## Define a Vocabulary

It is important to define a vocabulary of known words when using a bag-of-words model.

The more words, the larger the representation of documents, therefore it is important to constrain the words to only those believed to be predictive. This is difficult to know beforehand and often it is important to test different hypotheses about how to construct a useful vocabulary.

We have already seen how we can remove punctuation and numbers from the vocabulary in the previous section. We can repeat this for all documents and build a set of all known words.

We can develop a vocabulary as a Counter, which is a dictionary mapping of words and their count that allows us to easily update and query.

Each document can be added to the counter (a new function called add_doc_to_vocab()) and we can step over all of the reviews in the negative directory and then the positive directory (a new function called process_docs()).

## First lets create vocabulary

## Second Part

In [1]:
No_of_Files_to_process = 100

In [2]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from pandas import DataFrame
from matplotlib import pyplot


Using TensorFlow backend.


In [3]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding="latin1")
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

In [5]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)

In [6]:
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
    lines = list()
    count=0
    for filename in listdir(directory):
        count = count + 1
        if count <= No_of_Files_to_process :
            # create the full path of the file to open
            path = directory + '/' + filename
            # load and clean the doc
            line = doc_to_line(path, vocab)
            # add to list
            lines.append(line)
    return lines

In [7]:
# evaluate a neural network model
def evaluate_mode(Xtrain, ytrain, Xtest, ytest):
	scores = list()
	n_repeats = 1
	n_words = Xtest.shape[1]
	for i in range(n_repeats):
		# define network
		model = Sequential()
		model.add(Dense(50, input_shape=(n_words,), activation='relu'))
		model.add(Dense(1, activation='sigmoid'))
		# compile network
		model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
		# fit network
		model.fit(Xtrain, ytrain, epochs=50, verbose=2)
		# evaluate
		loss, acc = model.evaluate(Xtest, ytest, verbose=0)
		scores.append(acc)
		print('%d accuracy: %s' % ((i+1), acc))
	return scores

In [8]:
# prepare bag of words encoding of docs
def prepare_data(train_docs, test_docs, mode):
	# create the tokenizer
	tokenizer = Tokenizer()
	# fit the tokenizer on the documents
	tokenizer.fit_on_texts(train_docs)
	# encode training data set
	Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
	# encode training data set
	Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
	return Xtrain, Xtest


In [9]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# vocab

In [10]:
# load all training reviews
train_positive_lines = process_docs('data_movie_reviews/train/pos', vocab, True)
train_negative_lines = process_docs('data_movie_reviews/train/neg', vocab, True)
train_docs = train_positive_lines + train_negative_lines


In [11]:
# load all training reviews
test_positive_lines = process_docs('data_movie_reviews/test/pos', vocab, True)
test_negative_lines = process_docs('data_movie_reviews/test/neg', vocab, True)
test_docs = test_positive_lines + test_negative_lines


In [12]:
len(train_positive_lines), len(train_negative_lines), len(train_docs)

(100, 100, 200)

- ### 0 - Negative 
- ### 1 - Positive
- Create labels for each review

In [13]:
# prepare labels
ytrain = array([0 for _ in range(100)] + [1 for _ in range(100)])
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])

### Comparing Word Scoring Methods
The texts_to_matrix() function for the Tokenizer in the Keras API provides 4 different methods for scoring words; they are:

    “binary” Where words are marked as present (1) or absent (0).
    “count” Where the occurrence count for each word is marked as an integer.
    “tfidf” Where each word is scored based on their frequency, where words that are common across all documents are penalized.
    “freq” Where words are scored based on their frequency of occurrence within the document.
We can evaluate the skill of the model developed in the previous section fit using each of the 4 supported word scoring modes.

Epoch 1/50
 - 1s - loss: 0.6857 - acc: 0.5700
Epoch 2/50
 - 0s - loss: 0.4392 - acc: 0.9850
Epoch 3/50
 - 0s - loss: 0.2795 - acc: 1.0000
Epoch 4/50
 - 0s - loss: 0.1671 - acc: 1.0000
Epoch 5/50
 - 0s - loss: 0.1010 - acc: 1.0000
Epoch 6/50
 - 0s - loss: 0.0630 - acc: 1.0000
Epoch 7/50
 - 0s - loss: 0.0416 - acc: 1.0000
Epoch 8/50
 - 0s - loss: 0.0296 - acc: 1.0000
Epoch 9/50
 - 0s - loss: 0.0224 - acc: 1.0000
Epoch 10/50
 - 0s - loss: 0.0176 - acc: 1.0000
Epoch 11/50
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 12/50
 - 0s - loss: 0.0120 - acc: 1.0000
Epoch 13/50
 - 0s - loss: 0.0103 - acc: 1.0000
Epoch 14/50
 - 0s - loss: 0.0090 - acc: 1.0000
Epoch 15/50
 - 0s - loss: 0.0079 - acc: 1.0000
Epoch 16/50
 - 0s - loss: 0.0070 - acc: 1.0000
Epoch 17/50
 - 0s - loss: 0.0063 - acc: 1.0000
Epoch 18/50
 - 0s - loss: 0.0057 - acc: 1.0000
Epoch 19/50
 - 0s - loss: 0.0052 - acc: 1.0000
Epoch 20/50
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 21/50
 - 0s - loss: 0.0043 - acc: 1.0000
Epoch 22/50
 - 0s - lo

In [14]:
modes = ['binary', 'count', 'tfidf', 'freq']
results = DataFrame()
# for mode in modes:
for mode in ['binary']:
    # prepare data for mode
    Xtrain, Xtest = prepare_data(train_docs, test_docs, mode)
    # evaluate model on data for mode
    results[mode] = evaluate_mode(Xtrain, ytrain, Xtest, ytest)
# summarize results
print(results.describe())
# plot results
results.boxplot()
pyplot.show()